### ASTR 8070: Astrostatistics
***S. R. Taylor***
___

# Homework 4
### Due: Saturday, Feb 24th at 11.59pm CST
---

## Problem 1

This problem uses a dataset in `/coursework/homeworks/hw_data/`.

1) Read in `hw4_data_1.npy`. This is a (10 x 2) numpy array, with voltage measurements in the first column and heteroscedastic voltage uncertainties in the second column. Compute the sample mean and the standard error on the sample mean for this data.

2) Fit the appropriate ln-likelihood function and find the best-fit mean voltage.

3) Compute and plot the Bayesian posterior probability density (*not the log posterior*) for the mean voltage assuming a uniform prior for the mean in the range 3 to 7. Make sure this posterior pdf is normalized!

4) By either drawing samples from this posterior, or using your gridded posterior pdf to make a cdf, find the equal-tailed 68.3% credible region for the mean, and compare the upper and lower boundaries to the sample mean plus/minus the standard error, respectively. *Also* find the MAP value of the mean.

5) Repeat (3) and (4) this time with a prior on the mean that is uniform in the range 4.6 to 5.4. 

6) Now, imagine that we read an old paper about the experiment that gave us the voltage measurements, and they found that the mean was actually $6\pm0.3$. Repeat (3) and (4) this time with a Gaussian prior on the mean centered at $6$ with standard deviation of $0.3$.

7) Plot all of the normalized posterior pdfs for $\mu$ from (3), (5), and (6) on the same plot, making sure that the xlim of the plot spans 0 to 10.

8) You have made sure that the posterior pdfs are properly normalized, but until now you have ignored the meaning of that normalization constant. It is the Bayesian evidence for the particular model you have applied! Compute the evidence under a new model where the prior for the mean is a delta function at the best-fit value you found in (1) *(think about this and don't just immediately go looking for a `scipy.stats` delta function)*. Compare this to the evidence found under the prior in (3). Taking ratios to make a Bayes factor, which model is favored? Is there much of an Occam penalty by having the wide prior compared to knowing the mean exactly? 

### Solution

In [2]:
import numpy as np 
import scipy
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker
import chainconsumer
import corner

# 1.1

In [3]:
# LSS loading data
data1 = np.load('../../../homeworks/hw_data/hw4_data_1.npy')

def het_mean_sigma_analytic(data):
    '''
    A function to analytically calculate the mean and uncert on mean for 
    a heteroscedastic dataset

    inputs:
    data (ndarray): N measurements x 2 array where [:, 1] are the uncertainties

    returns:
    mean (float): mean analytically calculated for heteroscedastic data
    sigmu (float): uncertainty on the mean analytically calc.
    '''
    numerator = np.sum(data[:, 0] / data[:, 1]**2) # LSS sum of measurement over
    # the square of the respective uncertainties
    denom = np.sum(data[:, 1]**-2) # LSS sum of the inverse square of the uncert.

    mean = numerator / denom

    sigmu = denom**-0.5 # LSS uncert on mean is denominator to the -0.5
    return mean, sigmu

data1mu_an, data1sig_an = het_mean_sigma_analytic(data1)


In [8]:
data1mu_an

4.942118214425304

# 1.2

In [4]:
def neglnlik(mu_model, data, uncert, modelname='gaussian', scaleparam=np.nan):
    '''
    A function to return the negative log likelihood for data when modeled by a 
    gaussian with mu=mu_model

    inputs:
    mu_model (float): mean for the gaussian model
    data (ndarray): the data to be fit to 
    uncert (ndarray): the uncert for each data point
    outputs:
    neglnlik (float): negative log likelihood for some mu_model 
    '''
    if modelname=='gaussian':
        # LSS calculating likelihood
        neglnlik = np.sum(np.log(1/(uncert*np.sqrt(2*np.pi))) - ((data - mu_model)**2 / (2*uncert**2)))
    
    elif modelname=='laplace': # LSS encoding laplace model
        if np.isnan(scaleparam):
            print('MUST ASSIGN SCALE PARAMETER IF USING LAPLACE MODEL')
            return
        neglnlik = np.sum(np.log(1/(2*scaleparam))+-(np.abs(data-mu_model)/scaleparam))

    return -neglnlik

In [13]:
# LSS testing neglnlik function
neglnlik(4, data1[:, 0], data1[:, 1])

# LSS finding max ln lik (min negative)
nlnlik_mu1guess = lambda muguess: neglnlik(muguess, data=data1[:, 0], 
                                          uncert=data1[:, 1])
muguess0 = 3
mu1_MLE = optimize.fmin(nlnlik_mu1guess, muguess0)

Optimization terminated successfully.
         Current function value: 16.048013
         Iterations: 17
         Function evaluations: 34


In [16]:
print(f'MLE estimate: {mu1_MLE[0]}')
print(f'analytic est: {data1mu_an}')

MLE estimate: 4.942089843750004
analytic est: 4.942118214425304


## Problem 2

This problem uses a dataset in `/coursework/homeworks/hw_data/`.

1) Read in `hw4_data_2.npy`, which is a (3 x 20) numpy array that you used in `Lecture_9`. Set `x, y, sigma_y = data`. 

We're going to do some polynomial fits to this data just like in `Lecture 9`. However, in all cases you should **keep the $y$-intercept fixed at $-0.23$**. 

2) Use the following code to compute the un-normalized posterior pdf (i.e. just the likelihood x prior) on a grid of the linear coefficient (i.e. the slope) of a linear model, with a uniform prior between 0.5 and 1.5. Plot this posterior pdf. Remember this is just a one-dimensional model because the $y$-intercept is fixed. I advise a grid size of 100.

In [178]:
#Functions to do a polynomial fit, and compute the likelihood
def polynomial_fit(theta, x):
    """Polynomial model of degree (len(theta) - 1)"""
    # For a polynomial with order 1, this gives theta_0 + theta_1*x
    # For a polynomial with order 2, this gives theta_0 + theta_1*x + theta_2*x^2, etc.
    return sum(t * x ** n for (n, t) in enumerate(theta))

# compute the data log-likelihood given a model
def logL(theta, data, model=polynomial_fit):
    """Gaussian log-likelihood of the model at theta"""
    x, y, sigma_y = data
    y_fit = model(theta, x)
    return sum(scipy.stats.norm.logpdf(*args) 
               for args in zip(y, y_fit, sigma_y))

3) Using your 1D gridded likelihood-x-prior, compute the Bayesian evidence of this linear model. This may be a big number!

4) Now compute the joint 2D posterior pdf (again just the likelihood x prior) of linear and quadratic coefficients of a quadratic model. Give the linear coefficient a uniform prior between 0.5 and 1.5. Give the quadratic coefficient a uniform prior between -1 and 0.25. Plot this two-dimensional posterior. Remember this is a two-dimensional model because the $y$-intercept is fixed. I advise a grid size of 100 in each model dimension.

5) Using your 2D gridded likelihood-x-prior, compute the Bayesian evidence of the quadratic model. 

6) Calculate the Bayes factor for a linear versus quadratic model. How does this compare/contrast with the BIC model comparison in the lecture? 

### Solution